In [14]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os

In [15]:
# Read the data of the coin
coin_csv = "data/adausdt.csv"
coin = pd.read_csv(coin_csv)

# Check the data
coin.head(5)

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,LogReturns
0,1609459200000,0.18134,0.18146,0.17831,0.18051,19194919.1,1609462799999,3.447726e+06,5472,9753882.0,1.752001e+06,0.000000
1,1609462800000,0.18055,0.18469,0.18000,0.18358,28601648.4,1609466399999,5.222490e+06,11417,15629558.8,2.855128e+06,0.016864
2,1609466400000,0.18361,0.18473,0.18229,0.18368,20075571.1,1609469999999,3.692176e+06,6751,10958611.4,2.016056e+06,0.000545
3,1609470000000,0.18375,0.18458,0.18259,0.18292,15704497.1,1609473599999,2.886688e+06,5212,8379022.4,1.540624e+06,-0.004146
4,1609473600000,0.18299,0.18367,0.18105,0.18180,16338316.2,1609477199999,2.981119e+06,6630,7767811.4,1.417107e+06,-0.006142


In [7]:
# Check the shape of the coin dataframe
print(coin.shape)

(33561, 12)


In [11]:
# 1704067200000 is Jan 1, 2024
# 1711929600000 is Apr 1, 2024

training_benchmark = 1704067200000
validation_benchmark = 1711929600000

training_data = coin[coin["Open time"] < training_benchmark]
validation_data = coin[(coin["Open time"] < validation_benchmark) & (coin["Open time"] >= training_benchmark)]
testing_data = coin[coin["Open time"] >= validation_benchmark]

# Print the shape of each data
print(training_data.shape)
print(validation_data.shape)
print(testing_data.shape)


(26241, 12)
(2184, 12)
(5136, 12)


In [13]:
# Delete unnecessary columns
training_data_important = training_data.drop(["Open time", "Close time", "Number of trades", "Taker buy base asset volume", 'Taker buy quote asset volume', "Quote asset volume", "Volume"], axis = 1)
validation_data_important = validation_data.drop(["Open time", "Close time", "Number of trades", "Taker buy base asset volume", 'Taker buy quote asset volume', "Quote asset volume", "Volume"], axis = 1)
testing_data_important = testing_data.drop(["Open time", "Close time", "Number of trades", "Taker buy base asset volume", 'Taker buy quote asset volume', "Quote asset volume", "Volume"], axis = 1)

# Check the shape
print(training_data_important.shape)
print(validation_data_important.shape)
print(testing_data_important.shape)

(26241, 5)
(2184, 5)
(5136, 5)


In [ ]:
# Start first with training_data_important

# Check if there is already an image folder
image_folder = "image" 
if not os.path.exists(image_folder):
    os.makedirs(image_folder)

# After that, create a folder for each coin inside image_folder
coin_folder = coin_csv.split('/')[-1].replace('.csv', '')
coin_folder_path = os.path.join(image_folder, coin_folder)

if not os.path.exists(coin_folder_path):
    os.makedirs(coin_folder_path)

